In [ ]:
!pip install python-chi python-blazarclient python-dotenv

In [ ]:
# if running on local system (not chameleon jupyter)
from dotenv import load_dotenv
import chi
load_dotenv(".env")


In [ ]:
from chi import context
context.check_credentials()

In [ ]:
chi.set('project_name', 'CH-23118')
# Select your site
chi.use_site('CHI@TACC')

In [ ]:
from chi import lease
import datetime

l = lease.Lease(
    name="mlsys_gigaio_gpu",
    duration=datetime.timedelta(hours=6),
)

# Correct for GigaIO GPU pool
l.add_node_reservation(
    amount=1,
    node_type="compute_gigaio"
)

l.submit(idempotent=True)
l.add_fip_reservation(amount=1)


In [ ]:
from chi import server
s = server.Server('mlsys-bloat-mr7375',
                  reservation_id=l.node_reservations[0]["id"],
                  image_name="CC-Ubuntu24.04-CUDA",
                  flavor_name="baremetal",
                  key_name="sparkle",
                  network_name="sharednet1"
                  
)
s.submit(idempotent=True)


In [ ]:
import os
connect_kwargs = {"key_filename": os.getenv("KEY_PATH")}

In [ ]:
s.execute("sudo apt update", connect_kwargs=connect_kwargs)


In [ ]:
s.execute("curl -sSL https://get.docker.com/ | sudo sh", connect_kwargs=connect_kwargs)
s.execute("sudo groupadd -f docker; sudo usermod -aG docker $USER", connect_kwargs=connect_kwargs)
     

s.execute("curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg \
  && curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
    sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
    sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list", connect_kwargs=connect_kwargs)
s.execute("sudo apt update", connect_kwargs=connect_kwargs)
s.execute("sudo apt-get install -y nvidia-container-toolkit", connect_kwargs=connect_kwargs)
s.execute("sudo nvidia-ctk runtime configure --runtime=docker", connect_kwargs=connect_kwargs)
# for https://github.com/NVIDIA/nvidia-container-toolkit/issues/48
s.execute("sudo jq 'if has(\"exec-opts\", connect_kwargs=connect_kwargs) then . else . + {\"exec-opts\": [\"native.cgroupdriver=cgroupfs\"]} end' /etc/docker/daemon.json | sudo tee /etc/docker/daemon.json.tmp > /dev/null && sudo mv /etc/docker/daemon.json.tmp /etc/docker/daemon.json", connect_kwargs=connect_kwargs)
s.execute("sudo systemctl restart docker", connect_kwargs=connect_kwargs)


s.execute("sudo apt update", connect_kwargs=connect_kwargs)
s.execute("sudo apt -y install nvtop", connect_kwargs=connect_kwargs)

In [ ]:
s.execute("git clone --recurse-submodules https://github.com/meghrathod/mlsys-debloat", connect_kwargs=connect_kwargs)

In [ ]:
s.execute("sudo apt install -y python3-pip libspdlog-dev cmake build-essential curl pkg-config libssl-dev", connect_kwargs=connect_kwargs)
s.execute("curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y --default-toolchain stable", connect_kwargs=connect_kwargs)
